# PRODUCTO INTEGRADOR DE APRENDIZAJE
## Andrea Carolina López Ruiz
## José Alfonso González Acosta
## Leoplodo Hernandez Salazar 
### Estadística Aplicada Gpo 043
#### FCFM - Universidad Autónoma de Nuevo León
- 14 de mayo de 2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.arima.model import ARIMA

In [3]:
pas_met = pd.read_csv('metrorrey.csv')

In [4]:
pas_met

,Mes,Pasajeros,Ingresos
0,2008/01,5869.000,16979.000
1,2008/02,6499.000,18516.000
2,2008/03,5801.000,16719.000
3,2008/04,6932.000,17663.000
4,2008/05,6876.000,21079.000
...,...,...,...
176,2022/09,13244.858,45671.599
177,2022/10,14135.829,47422.015
178,2022/11,13657.740,44794.633
179,2022/12,10665.017,41252.317
